<a href="https://colab.research.google.com/github/RaghavGanesh7/Coding-AIAgents/blob/main/Langgraph_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install langgraph

# New Section

In [ ]:
from typing import TypedDict,List
from langgraph.graph import StateGraph, START, END

In [ ]:
class AgentState(TypedDict):
    name:str
    values: List[int]
    result : str

In [ ]:
def process_values(state:AgentState):
    #print(state)
    """function to process multiple inputs"""
    state['result'] = "Hi " + state['name'] + ". Your sum is " + str(sum(state['values']))
    return state

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("processor", process_values)
graph.set_entry_point("processor")
graph.set_finish_point("processor")

app = graph.compile()

In [ ]:
app.invoke({"name":"Raghav","values":[1,2,3,4]})['result']

'Hi Raghav. Your sum is 10'

In [ ]:
class RouterState(TypedDict):
  operator : str
  num1 : int
  num2 : int
  result : str

In [ ]:
def add_operation(state: RouterState):
  print(state)
  state['result'] = str(state['num1'] + state['num2'])
  return state

def subtract_operation(state:RouterState):
  state['result'] = str(state['num1'] - state['num2'])
  return state

def myrouter(state:RouterState):
  print(state)
  if state['operator'] == '+':
    return "add_operation"
  elif state['operator'] == '-':
    return "subtract_operation"

In [ ]:
graph = StateGraph(RouterState)
graph.add_node("add_node", add_operation)
graph.add_node("sub_node", subtract_operation)
graph.add_node("router", lambda state:state)

graph.add_edge(START, "router");
graph.add_conditional_edges("router",myrouter,{
    "add_operation": "add_node",
    "subtract_operation": "sub_node"
})
graph.add_edge("add_node", END)
graph.add_edge("sub_node",END)

app = graph.compile()


In [ ]:
initState = RouterState(operator='+',num1=1,num2=2);
app.invoke(initState)['result']

{'operator': '+', 'num1': 1, 'num2': 2}
{'operator': '+', 'num1': 1, 'num2': 2}


'3'

In [ ]:
class AgentState(TypedDict):
  player_name:str
  target: int
  guesses: List[int]
  attempts: int
  lower: int
  upper: int

In [ ]:
def setup(state:AgentState):
  state['attempts'] = 0
  state['lower'] = 0
  state['upper'] = 100
  state['guesses'] = []
  return state
def guess(state:AgentState):
  state['attempts'] += 1
  state['guesses'].append((state['lower'] + state['upper']) // 2)
  return state
def hint_node(state:AgentState):
  print(state)
  if state['guesses'][-1] < state['target']:
    state['lower'] = state['guesses'][-1]
  elif state['guesses'][-1] > state['target']:
    state['upper'] = state['guesses'][-1]
  return state
def should_continue(state:AgentState):
  if state['guesses'][-1] == state['target']:
    return "continue"
  elif state['attempts'] > 10:
    return "continue"
  else:
    return "loop"

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("setup",setup)
graph.add_node("loop",guess)
graph.add_node("hint",hint_node)
graph.add_node("should_continue",lambda x:x)

graph.add_edge(START,"setup")
graph.add_edge("setup","loop")
graph.add_edge("loop","hint")
graph.add_conditional_edges("hint",should_continue,{
    "continue": END,
    "loop": "loop"
})
graph.add_edge("hint",END)

app = graph.compile()
init_state = AgentState(target=60,player_name="Raghav")
print(app.invoke(init_state))

{'player_name': 'Raghav', 'target': 60, 'guesses': [50], 'attempts': 1, 'lower': 0, 'upper': 100}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75], 'attempts': 2, 'lower': 50, 'upper': 100}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62], 'attempts': 3, 'lower': 50, 'upper': 75}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56], 'attempts': 4, 'lower': 50, 'upper': 62}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56, 59], 'attempts': 5, 'lower': 56, 'upper': 62}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56, 59, 60], 'attempts': 6, 'lower': 59, 'upper': 62}
{'player_name': 'Raghav', 'target': 60, 'guesses': [50, 75, 62, 56, 59, 60], 'attempts': 6, 'lower': 59, 'upper': 62}


In [23]:
!pip install langgraph

!pip install langchain_google_genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict,List,Union
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from google.colab import userdata

In [ ]:
class AgentState(TypedDict):
  messages: List[Union[HumanMessage,AIMessage]]

llm  = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=userdata.get('GOOGLE_API_KEY'))


def ex_process(state:AgentState):
  response = llm.invoke(state['messages'])
  state['messages'].append(AIMessage(content=response.content))
  print(response.content)
  return state

graph = StateGraph(AgentState)
graph.add_node("ex_process", ex_process)
graph.add_edge(START,"ex_process")
graph.add_edge("ex_process",END)

agent = graph.compile()

conversation_history = []

user_input = input("Enter :")

while(user_input != "exit"):
  conversation_history.append(HumanMessage(content=user_input))
  state = agent.invoke({"messages":conversation_history})
  conversation_history = state['messages']
  user_input = input("Enter :")
  #agent.invoke({"message":[HumanMessage(content="Hi")]})

Enter :Hi
Hi there! How can I help you today?
Enter :My name is Raghav
Nice to meet you, Raghav! It's good to have you here.

How can I assist you today, Raghav?
Enter :What is my name?
Your name is Raghav.
Enter :What was first message?
My first message to you was: "Hi there! How can I help you today?"
Enter :quit
Okay, Raghav. It was nice chatting with you!

If you need anything else in the future, feel free to reach out. Have a great day!
Enter :exit


In [24]:
!pip install langchain

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated,Sequence,TypedDict,List,Union
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import BaseMessage
from langchain_core.messages import ToolMessage
from langchain_core.messages import SystemMessage
from google.colab import userdata

In [28]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

@tool
def add(a:int, b:int):
  """This is a addition function to add two numbers"""
  return a+b

def multiply(a:int, b:int):
  """This is a multiplication function to multiply two numbers"""
  return a*b

tools = [add,multiply]
llm  = ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=userdata.get('GOOGLE_API_KEY')).bind_tools(tools)

def mode_call(state:AgentState):
  system_prompt = SystemMessage(content="You a AI Agent. Answer to best of your ability")
  response = llm.invoke([system_prompt] + state["messages"])
  print(response.content)
  return {'messages': [response]}

def should_continue(state:AgentState):
  messages = state['messages']
  last_message = messages[-1]
  if not last_message.tool_calls:
    return "end"
  else:
    return "continue"


graph = StateGraph(AgentState)
graph.add_node("mode_call", mode_call)
tool_node = ToolNode(tools=tools)
graph.add_node("tools",tool_node)
graph.add_conditional_edges("mode_call",should_continue,{
    "continue": "tools",
    "end": END
})
graph.set_entry_point("mode_call")
graph.add_edge("tools","mode_call")

app = graph.compile()

def print_stream(stream):
  for s in stream:
    message = s['messages'][-1]
    if isinstance(message, tuple):
      print(message)
    else:
      message.pretty_print()

inputs = {"messages":[("user","Add 5+5. And then multiply the result with 5. Also who created you?")]}

print_stream(app.stream(inputs,stream_mode="values"))

================================ Human Message =================================

Add 5+5. And then multiply the result with 5. Also who created you?

================================== Ai Message ==================================
Tool Calls:
  add (f644f403-d1fd-4425-83d2-b56ba3e4f9b1)
 Call ID: f644f403-d1fd-4425-83d2-b56ba3e4f9b1
  Args:
    a: 5
    b: 5
================================= Tool Message =================================
Name: add

10

================================== Ai Message ==================================
Tool Calls:
  multiply (f7eed326-8831-4daa-8ea3-61b349db9102)
 Call ID: f7eed326-8831-4daa-8ea3-61b349db9102
  Args:
    a: 10
    b: 5
================================= Tool Message =================================
Name: multiply

50
I am a large language model, trained by Google.
================================== Ai Message ==================================

I am a large language model, trained by Google.


In [ ]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage],add_messages]